In [ ]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold
from time import time

In [ ]:
df = pd.read_csv('WDI_csv/WDIData.csv',index_col='Country Name')
df = df.iloc[:, :-1]
print(df.shape)

In [ ]:
set(df.loc['Afghanistan']['Indicator Name'])

In [ ]:
print(len(set(df.index)))

In [ ]:
from collections import Counter
a = df[df['1962'].notnull()]['Indicator Name']
test = Counter()
for b, c in zip(a, a.index):
    test[b] += 1

    

In [ ]:
test.most_common(10)

In [ ]:
datas = df[df['Indicator Name'] == 'GDP per capita (current US$)']
datas

In [ ]:
datas = datas.iloc[:, 3:]
datas

In [ ]:
from scipy.interpolate import interp1d, CubicSpline, PchipInterpolator
from scipy.optimize import curve_fit
import math
import copy

def func_linear(x, a, b):
    return a + b * x

def func_cubic(x, a, b, c, d):
    return a + b * x + c * x**2 + d * x**3

datas_copy = copy.deepcopy(datas)

for i in range(datas.shape[0]):
# for i in range(0, 1):
    original_row = datas.iloc[i:i+1,:]
    curated_row = original_row.dropna(axis='columns', how='any')
    
    if len(curated_row.values[0]) == 0:
        continue
        
    original_row = original_row.iloc[0]
    curated_row = curated_row.iloc[0]
    
    indexes = original_row.index.values
    indexes = indexes.astype(np.float)

    x = curated_row.index.values
    x = x.astype(np.float)
    y = curated_row.values
    
    f = interp1d(x, y, kind='linear', fill_value='extrapolate')
    cubicspline = CubicSpline(x, y, extrapolate='None')
    Pchip = PchipInterpolator(x, y, extrapolate='True')
    
    z = np.polyfit(x, y, 4)
    p = np.poly1d(z)
    
    popt_linear, _ = curve_fit(func_linear, x, y)
    popt_cubic, _ = curve_fit(func_cubic, x, y)
    
    flag = False
    for n in range(len(indexes)):
        if math.isnan(datas_copy.iat[i,n]):
            value = Pchip(indexes[n])
            if(value < 0):
                print(value)
                value = 0
            datas_copy.iat[i,n] = value
            flag = True
    if flag:
        print(datas_copy.iloc[i:i+1,:].values)

In [ ]:
xnew = np.linspace(1960, 2017, endpoint=True)
plt.plot(x, y, 'o', xnew, p(xnew), '-', xnew, cubicspline(xnew), '--', xnew, Pchip(xnew), '-', 
         xnew, func_linear(xnew, *popt_linear), 'r-', xnew, func_cubic(xnew, *popt_cubic), 'b-')
plt.legend(['data', 'polyfit interpolation', 'cubicspline interpolation', 'Pchip interpolation',
            'linear function approximate', 'cubic function approximate'], loc='best')
plt.show()

In [ ]:
datas = datas.iloc[:, 3:]
datas = datas.dropna(axis='columns', how='all')
datas = datas.dropna(axis='index', how='any')
X = datas.values

In [ ]:
datas.index

In [ ]:
from dtaidistance import dtw
from sklearn.preprocessing import normalize
X = normalize(X, norm='max')
ds = dtw.distance_matrix_fast(X)
ds[ds == np.inf] = 0
ds

In [ ]:
from dtaidistance import clustering
# Custom Hierarchical clustering
model1 = clustering.Hierarchical(dtw.distance_matrix_fast, {})
# Augment Hierarchical object to keep track of the full tree
model2 = clustering.HierarchicalTree(model1)
# SciPy linkage clustering
model3 = clustering.LinkageTree(dtw.distance_matrix_fast, {})
cluster_idx1 = model3.fit(datas.values[:20,:])

# cluster_idx2 = model2.fit(datas.values)

In [ ]:

fig, ax = plt.subplots(figsize=(8,6))
t0 = time()
mds = manifold.TSNE(2, init='pca', random_state=0)
Y = mds.fit_transform(ds)
t1 = time()
color = [i for i in range(len(Y))]
ax.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.Spectral)
plt.title("TSNE (%.2g sec)" % (t1 - t0))

for i, txt in enumerate(datas.index):
    ax.annotate(txt, (Y[i, 0], Y[i, 1]))
plt.show()

In [ ]:
import pyximport; pyximport.install()
from frechet import frechet

In [ ]:
P = datas.values[0].copy(order='C')
Q = datas.values[1].copy(order='C')
frechet(P, Q)

In [ ]:
Y